In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
rr_weight_layer_two_pre = pd.read_csv("results/rr_s_weight_pre_3.csv").to_numpy()
rr_weight_layer_two_post = pd.read_csv("results/rr_s_weight_post_3.csv").to_numpy()
ss_weight_layer_two_pre = pd.read_csv("results/ss_s_weight_pre_3.csv").to_numpy()
ss_weight_layer_two_post = pd.read_csv("results/ss_s_weight_post_3.csv").to_numpy()
rr_footprint = rr_weight_layer_two_post - rr_weight_layer_two_pre
ss_footprint = ss_weight_layer_two_post - ss_weight_layer_two_pre


In [25]:
pool = nn.MaxPool1d(3, stride=2)
pooled_rr_footprint = pool(torch.from_numpy(rr_footprint))
print(pooled_rr_footprint.numpy().shape)

(10, 255)


In [3]:
#Import the dataset

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512,255),
            nn.Sigmoid(),
            nn.Linear(255, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=512, out_features=255, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=255, out_features=10, bias=True)
  )
)


In [34]:
#2 layer 37 epoch
#2 Layer rr_footprint: 30
#2 layer rr_layer: 27 epoch
t, acc = 0, 0
while acc < 80:
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    acc = test(test_dataloader, model, loss_fn) * 100
    t += 1
print("Done!")

Epoch 1
-------------------------------
loss: 3.410338  [   64/60000]
loss: 2.300101  [ 6464/60000]
loss: 2.298646  [12864/60000]
loss: 2.293285  [19264/60000]
loss: 2.268168  [25664/60000]
loss: 2.283952  [32064/60000]
loss: 2.275866  [38464/60000]
loss: 2.258720  [44864/60000]
loss: 2.258068  [51264/60000]
loss: 2.270478  [57664/60000]
Test Error: 
 Accuracy: 20.2%, Avg loss: 2.251863 

Epoch 2
-------------------------------
loss: 2.257909  [   64/60000]
loss: 2.246567  [ 6464/60000]
loss: 2.235643  [12864/60000]
loss: 2.229307  [19264/60000]
loss: 2.168821  [25664/60000]
loss: 2.185651  [32064/60000]
loss: 2.148215  [38464/60000]
loss: 2.119588  [44864/60000]
loss: 2.077099  [51264/60000]
loss: 2.046587  [57664/60000]
Test Error: 
 Accuracy: 37.4%, Avg loss: 2.044736 

Epoch 3
-------------------------------
loss: 2.058706  [   64/60000]
loss: 2.017439  [ 6464/60000]
loss: 1.973233  [12864/60000]
loss: 1.953409  [19264/60000]
loss: 1.806508  [25664/60000]
loss: 1.846423  [32064/600

In [33]:
new_state = model.state_dict()
new_state["linear_relu_stack.4.weight"] = pooled_rr_footprint
model.load_state_dict(new_state)

<All keys matched successfully>